In [ ]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras import optimizers

In [ ]:
#Mouting google drive to access data 
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    print("Mounted With Google Drive")
    COLAB = True
    %tensorflow_version 2.x
except:
    COLAB = False

Mounted at /content/drive
Mounted With Google Drive


In [ ]:
#We have stored data in My drive/agan folder
DATA_PATH = '/content/drive/My Drive/Dataset'

In [ ]:
#Our test and train directory have specified below
train_data_dir  = '/content/drive/My Drive/Dataset/train'
validation_data_dir  = '/content/drive/My Drive/Dataset/test'

In [ ]:
#Configuring the dataset

#Batch size is 10(Will take 10 image for each run)
batch_size = 10
#Epoch is 50(Will run 50times)
epochs = 10
#Image heigh and widh reduced here for faster train and test
img_height = 150
img_width = 150

In [ ]:
#Normalizing the train and test generator for our model
train_image_generator = ImageDataGenerator(rescale=1./255) 
validation_image_generator = ImageDataGenerator(rescale=1./255) 

In [ ]:
# used to rescale the pixel values from [0, 255] to [0, 1] interval
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=5,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(IMG_WIDTH, IMG_HEIGHT),
        batch_size=5,
        class_mode='binary')

Found 101 images belonging to 2 classes.
Found 12 images belonging to 1 classes.


In [ ]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(img_width, img_height,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))


model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
nb_epoch = 5
nb_train_samples = 5
nb_validation_samples = 5

In [ ]:
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples,
        epochs=nb_epoch,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
5/5 [==============================] - 7s 1s/step - loss: 0.6992 - accuracy: 0.4656 - val_loss: 0.7903 - val_accuracy: 0.0000e+00
Epoch 2/5
5/5 [==============================] - 6s 1s/step - loss: 0.6516 - accuracy: 0.6628
Epoch 3/5
5/5 [==============================] - 3s 500ms/step - loss: 0.6339 - accuracy: 0.7400
Epoch 4/5
5/5 [==============================] - 2s 530ms/step - loss: 0.6117 - accuracy: 0.6044
Epoch 5/5
5/5 [==============================] - 2s 281ms/step - loss: 0.6879 - accuracy: 0.6056


In [ ]:
model.evaluate_generator(validation_generator, nb_validation_samples)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.9466539025306702, 0.0]

In [ ]:
#Creating Train Data Generator with the selected configuration
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_data_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

In [ ]:
#Creating Test Data Generator with the selected configuration
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_data_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

In [ ]:
#Storing train_data_generator into simple_training_images
sample_training_images, _ = next(train_data_gen)

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
#Showing Training images including normal and COVID-19
import matplotlib.pyplot as plt
plotImages(sample_training_images[:10])

In [ ]:
#Creating our main CNN model to augment images
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3)),
    MaxPooling2D(),

    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),

    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),

    Flatten(),
    Dense(512, activation='relu'),
    
    Dense(1)
])

In [ ]:
#Adding ADAM optimizer to optimize output
#importing tensorflow
import tensorflow as tf
model.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
#Printing Model Summery
model.summary()

In [ ]:
#Training our model with 50 epoch


history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=50 // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=40 // batch_size
)

In [ ]:
#Showing accuracy and loss graph
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#Basic Augmentation
image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True,vertical_flip=True)

In [ ]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_data_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)
print("Diamond Potato")

In [ ]:
#Variation 3
image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True,vertical_flip=True,samplewise_center=True,samplewise_std_normalization=True)

In [ ]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_data_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(10)]

In [ ]:
plotImages(augmented_images)

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, rotation_range=45)

In [ ]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=validation_data_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

In [ ]:
plotImages(augmented_images)

In [ ]:
#Variation 2
image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True,vertical_flip=True)

In [ ]:
train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_data_dir,
                                               shuffle=True,
                                               target_size=(IMG_HEIGHT, IMG_WIDTH))

In [ ]:
plotImages(augmented_images)
print("JAM ALU")